In [1]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [2]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.13', ray_version='2.0.0rc0', ray_commit='a0588094ec52b45a878f59e98258cd5e90f4ec36', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-11_08-21-05_287406_99146/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-11_08-21-05_287406_99146/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-11_08-21-05_287406_99146', 'metrics_export_port': 62410, 'gcs_address': '127.0.0.1:64156', 'address': '127.0.0.1:64156', 'dashboard_agent_listen_port': 52365, 'node_id': '2feed46fbb732dd7526bb2acc8403479a0d3f43b03289f7aa15963c7'})

In [3]:
CALLERS = ["A", "B", "C"]

@ray.remote
class MethodStateCounter:
    def __init__(self):
        self.invokers = {"A": [0], "B": [0], "C": [0]}
    
    def invoke(self, name):
        # pretend to do some work
        time.sleep(0.5)
        
        # compute a random result and update the result
        result = random.randint(5, 25)
        self.invokers[name].append(result)
        return result
        
    def get_invoker_state(self, name):
        return self.invokers[name]
    
    def get_number_of_calls(self, name):
        return len(self.invokers[name])
    
    def get_all_invoker_state(self):
        return self.invokers

In [4]:
actor_instance = MethodStateCounter.remote()

In [5]:
for _ in range(10):
    name = random.choice(CALLERS)
    actor_instance.invoke.remote(name)

#### 1. Get number of times an invoker name was called

In [6]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_number_of_calls.remote(name))
print(f"Number of times: {name} was called: {count}")

Number of times: A was called: 5


#### 2. Get computed values by invoker name

In [7]:
name = random.choice(CALLERS)
count = ray.get(actor_instance.get_invoker_state.remote(name))
print(f"All values computed by: {name} is: {count}")

All values computed by: A is: [0, 5, 13, 17, 24]


#### 3. Get state of all invokers

In [8]:
print(f"State of all computed values for invokers: {ray.get(actor_instance.get_all_invoker_state.remote())}")

State of all computed values for invokers: {'A': [0, 5, 13, 17, 24], 'B': [0, 19, 25, 15, 6, 19], 'C': [0, 25]}


In [9]:
ray.shutdown()